In [1]:


import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, RandomForestClassifier,StackingClassifier,BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
import lightgbm as lgb
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, RobustScaler, OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, make_scorer,mean_squared_error
#from catboost import CatBoostClassifier, Pool, cv
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.metrics import roc_curve, roc_auc_score, auc
# import matplotlib.pyplot as plt
import lightgbm as lgb
import xgboost as xgb


In [29]:

train_set = pd.read_csv(r'C:\Users\hrahe\Downloads\train_set.csv')
test_set = pd.read_csv(r'C:\Users\hrahe\Downloads\test_set.csv')

In [30]:
categorical_cols = []
numerical_cols = []

for column in train_set.columns:
  if train_set[column].dtype == object or train_set[column].nunique() < 10:
    categorical_cols.append(column)
  else:
    numerical_cols.append(column)

print("Categorical Columns:", categorical_cols)
print("Numerical Columns:", numerical_cols)

Categorical Columns: ['X4', 'X5', 'X6', 'X8', 'X10', 'X11', 'X16', 'Y']
Numerical Columns: ['RecordId', 'X2', 'X3', 'X7', 'X9', 'X12', 'X13', 'X14', 'X15', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42', 'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52', 'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62', 'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72', 'X73', 'X74', 'X75', 'X76', 'X77', 'X78']


In [31]:
# Handle missing values for both train and test datasets
for column in categorical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mode_imputer = SimpleImputer(strategy='most_frequent')
    train_set[column] = mode_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mode_imputer = SimpleImputer(strategy='most_frequent')
      test_set[column] = mode_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")

for column in numerical_cols:
  if column in train_set.columns and train_set[column].isnull().any():
    mean_imputer = SimpleImputer(strategy='mean')
    train_set[column] = mean_imputer.fit_transform(train_set[[column]])
  if column in test_set.columns and test_set[column].isnull().any():
    if column in train_set.columns:
      mean_imputer = SimpleImputer(strategy='mean')
      test_set[column] = mean_imputer.fit_transform(test_set[[column]])
    else:
      print(f"Warning: Column '{column}' is missing in the training set and cannot be imputed in the test set.")


# Find missing values in the training set
missing_values = train_set.isnull().sum()
print(missing_values[missing_values > 0])

# Find missing values in the test set
missing_values = test_set.isnull().sum()
print(missing_values[missing_values > 0])



Series([], dtype: int64)
Series([], dtype: int64)


In [32]:


# Assuming train_set and test_set are pandas DataFrames
# Get all columns except 'Y' for X
X = train_set[[col for col in train_set.columns if col != 'Y']]

# Get only 'Y' column for y
y = train_set['Y']

# Select the same features for the test data
X_testdata = test_set[[col for col in test_set.columns if col != 'Y']]
if 'RecordId' in X.columns:
  X = X.drop('RecordId', axis=1)
if 'RecordId' in X_testdata.columns:
  X_testdata = X_testdata.drop('RecordId', axis=1)

# ... rest of your code (scaling, feature selection, model training, etc.) ...
X.columns

Index(['X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12',
       'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22',
       'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32',
       'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42',
       'X43', 'X44', 'X45', 'X46', 'X47', 'X48', 'X49', 'X50', 'X51', 'X52',
       'X53', 'X54', 'X55', 'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62',
       'X63', 'X64', 'X65', 'X66', 'X67', 'X68', 'X69', 'X70', 'X71', 'X72',
       'X73', 'X74', 'X75', 'X76', 'X77', 'X78'],
      dtype='object')

In [25]:
scalar=StandardScaler()
X = scalar.fit_transform(X)
X_testdata = scalar.transform(X_testdata)




In [ ]:
# import lightgbm as lgb
# import pandas as pd
# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import train_test_split


# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# # Initialize and train the LightGBM model on full feature set to get feature importances
# clf = lgb.LGBMClassifier(
#     objective='binary',
#     metric='binary_logloss',
#     boosting_type='gbdt',
#     num_leaves=20,
#     max_depth=4,
#     learning_rate=0.03,
#     n_estimators=376,
#     subsample=0.7,
#     colsample_bytree=0.5,
#     min_child_weight=5,
#     scale_pos_weight=1.1,
#     reg_lambda=0.1,
#     verbose=-1
# )

# # Fit the model on full data to get feature importances
# clf.fit(X_train, y_train)

# # Get feature importances and create a DataFrame
# feature_importances = clf.feature_importances_
# feature_importances_df = pd.DataFrame({
#     'Feature': X_train.columns,
#     'Importance': feature_importances
# })

# # Sort the DataFrame by importance
# feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

# # Define different numbers of top features to test
# feature_counts = [10, 15, 20, 25, 30,35,40,45,50,55,60]
# auc_scores = {}

# # Iterate over different numbers of top features
# for n in feature_counts:
#     # Select the top n features
#     top_features = feature_importances_df['Feature'].head(n).tolist()

#     # Subset the training and testing data to the top n features
#     X_train_top = X_train[top_features]
#     X_test_top = X_test[top_features]

#     # Re-train the model with the top n features
#     clf.fit(X_train_top, y_train)

#     # Predict probabilities on the test set
#     y_pred_proba = clf.predict_proba(X_test_top)[:, 1]

#     # Calculate AUC score
#     auc = roc_auc_score(y_test, y_pred_proba)
#     auc_scores[n] = auc  # Store the AUC score for this number of features

#     print(f"Top {n} features - AUC: {auc:.4f}")

# # Print all AUC scores for comparison
# print("\nAUC scores for different feature counts:")
# for n, auc in auc_scores.items():
#     print(f"Top {n} features: AUC = {auc:.4f}")


Top 10 features - AUC: 0.9437
Top 15 features - AUC: 0.9500
Top 20 features - AUC: 0.9586
Top 25 features - AUC: 0.9619
Top 30 features - AUC: 0.9608
Top 35 features - AUC: 0.9625
Top 40 features - AUC: 0.9632
Top 45 features - AUC: 0.9609
Top 50 features - AUC: 0.9622
Top 55 features - AUC: 0.9616
Top 60 features - AUC: 0.9640

AUC scores for different feature counts:
Top 10 features: AUC = 0.9437
Top 15 features: AUC = 0.9500
Top 20 features: AUC = 0.9586
Top 25 features: AUC = 0.9619
Top 30 features: AUC = 0.9608
Top 35 features: AUC = 0.9625
Top 40 features: AUC = 0.9632
Top 45 features: AUC = 0.9609
Top 50 features: AUC = 0.9622
Top 55 features: AUC = 0.9616
Top 60 features: AUC = 0.9640


In [26]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [33]:
import xgboost as xgb
import pandas as pd
from sklearn.ensemble import BaggingClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Assuming X and y are already defined
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# Scale the features using MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Select the top k features
k = 60  # Change this value to select a different number of top features
selector = SelectKBest(score_func=f_classif, k=k)
selector.fit(X_train_scaled, y_train)

# Get the selected feature names
selected_features = X_train.columns[selector.get_support()]
print("Selected features:", selected_features)

# Transform the datasets
X_train_selected = selector.transform(X_train_scaled)
X_test_selected = selector.transform(X_test_scaled)

# Initialize the XGBoost model
clf = xgb.XGBClassifier(
    subsample=0.8,
    reg_lambda=0.1,
    n_estimators=12120,
    min_child_weight=60,
    max_depth=4,
    learning_rate=0.0045,
    objective='binary:logistic',
    eval_metric='logloss',
    random_state=42
)

# Initialize the BaggingClassifier with the XGBoost classifier
bagging_clf = BaggingClassifier(base_estimator=clf, n_estimators=10, random_state=42)

# Re-train the model with the selected features
bagging_clf.fit(X_train_selected, y_train)

# Make predictions
y_pred_proba = bagging_clf.predict_proba(X_test_selected)[:, 1]

# Calculate AUC
auc = roc_auc_score(y_test, y_pred_proba)
print(f"AUC: {auc}")

# Make predictions for other metrics
y_pred = bagging_clf.predict(X_test_selected)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Selected features: Index(['X2', 'X4', 'X6', 'X8', 'X10', 'X12', 'X14', 'X15', 'X16', 'X17', 'X18',
       'X19', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X28', 'X30',
       'X31', 'X34', 'X36', 'X37', 'X38', 'X39', 'X40', 'X41', 'X42', 'X43',
       'X44', 'X46', 'X47', 'X48', 'X50', 'X51', 'X52', 'X53', 'X54', 'X55',
       'X56', 'X57', 'X58', 'X59', 'X60', 'X61', 'X62', 'X63', 'X64', 'X65',
       'X67', 'X68', 'X69', 'X70', 'X72', 'X74', 'X75', 'X77', 'X78'],
      dtype='object')


c:\Users\hrahe\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


XGBoostError: [16:55:28] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0ed59c031377d09b8-1\xgboost\xgboost-ci-windows\src\common\io.h:320: bad_malloc: Failed to allocate 11813820 bytes.

In [ ]:
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.tree import DecisionTreeClassifier

# # Define a smaller parameter grid for faster search
# param_grid = {
#     'n_estimators': [50, 100, 200],           # Reduce the range of estimators
#     'learning_rate': [0.05, 0.1, 0.5],        # Use a smaller set of learning rates
#     'estimator__max_depth': [1, 2, 3],        # Limit the max depth for faster results
#     'estimator__min_samples_split': [2, 5]    # Use fewer values for min_samples_split
# }

# # Initialize the AdaBoost classifier with a DecisionTree as the base estimator
# model = AdaBoostClassifier(
#     estimator=DecisionTreeClassifier(random_state=42),
#     random_state=42
# )

# # Set up RandomizedSearchCV with fewer iterations
# random_search = RandomizedSearchCV(
#     estimator=model,
#     param_distributions=param_grid,
#     cv=3,
#     n_iter=10,               # Reduced to 10 for faster search
#     scoring='roc_auc',
#     verbose=1,
#     n_jobs=-1,
#     random_state=42
# )

# # Fit the model
# random_search.fit(X_train, y_train)

# # Print the best parameters
# print("Best parameters found: ", random_search.best_params_)


Fitting 3 folds for each of 50 candidates, totalling 150 fits


In [ ]:
# # Retrieve the best parameters from the randomized search
# best_params = random_search.best_params_

# # Initialize a new AdaBoost model with the best parameters
# clf = AdaBoostClassifier(
#     base_estimator=DecisionTreeClassifier(
#         max_depth=best_params['base_estimator__max_depth'],
#         min_samples_split=best_params['base_estimator__min_samples_split'],
#         random_state=42
#     ),
#     n_estimators=best_params['n_estimators'],
#     learning_rate=best_params['learning_rate'],
#     random_state=42
# )

# # Fit the model with the best parameters on the training data
# clf.fit(X_train, y_train)

# # Optionally, evaluate the model on the training or validation set
# print("Model trained with best parameters.")


In [ ]:


# # Initialize the LightGBM model
# clf = lgb.LGBMClassifier(  objective= 'binary',
#     metric= 'binary_logloss',
#     boosting_type= 'gbdt',
#     num_leaves= 20,
#     max_depth= 5,
#     learning_rate= 0.03,
#     n_estimators= 500,
#     subsample= 0.7,
#     colsample_bytree= 0.5,
#     min_child_weight= 5,
#     scale_pos_weight= 1.1,
#     reg_lambda= 0.1,
#     verbose= -1)

# clf.fit(X_train, y_train)

LGBMClassifier(colsample_bytree=0.5, learning_rate=0.03, max_depth=5,
               metric='binary_logloss', min_child_weight=5, n_estimators=500,
               num_leaves=20, objective='binary', reg_lambda=0.1,
               scale_pos_weight=1.1, subsample=0.7, verbose=-1)

In [8]:
import xgboost as xgb
import pandas as pd
from sklearn.metrics import roc_auc_score

# Assuming clf is already defined and trained
# Get feature importances
feature_importances = clf.feature_importances_

# Create a DataFrame for feature importances
feature_importances_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
})

# Sort the DataFrame by importance
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

# Print the feature importances
print(feature_importances_df)

# List of top feature counts to evaluate
top_feature_counts = [10, 20, 25, 30, 35, 40, 45, 60, 65, 70, 77]

# Dictionary to store AUC values for different feature counts
auc_values = {}

for count in top_feature_counts:
    # Select top features
    top_features = feature_importances_df['Feature'].head(count).tolist()
    
    # Select the top features from the datasets
    X_train_top = X_train[top_features]
    X_test_top = X_test[top_features]
    X_testdata_top = X_testdata[top_features]
    
    # Re-train the model with the top features
    clf.fit(X_train_top, y_train)
    
    # Make predictions
    y_pred_proba = clf.predict_proba(X_test_top)[:, 1]
    
    # Calculate AUC
    auc = roc_auc_score(y_test, y_pred_proba)
    auc_values[count] = auc
    print(f"AUC for top {count} features: {auc}")

# Print all AUC values
print("AUC values for different numbers of top features:")
for count, auc in auc_values.items():
    print(f"Top {count} features: AUC = {auc}")

   Feature  Importance
67     X69    0.069522
16     X18    0.041260
24     X26    0.036555
68     X70    0.032116
26     X28    0.027500
..     ...         ...
2       X4    0.000000
72     X74    0.000000
74     X76    0.000000
75     X77    0.000000
76     X78    0.000000

[77 rows x 2 columns]
AUC for top 10 features: 0.9511335898089008
AUC for top 20 features: 0.9539279219509883
AUC for top 25 features: 0.9604894193576021
AUC for top 30 features: 0.9583819939752696
AUC for top 35 features: 0.9586781726776514
AUC for top 40 features: 0.9608515547787067
AUC for top 45 features: 0.9616491868071981
AUC for top 60 features: 0.9634336634890489
AUC for top 65 features: 0.9637113310225319
AUC for top 70 features: 0.9636534053032391
AUC for top 77 features: 0.9636534053032391
AUC values for different numbers of top features:
Top 10 features: AUC = 0.9511335898089008
Top 20 features: AUC = 0.9539279219509883
Top 25 features: AUC = 0.9604894193576021
Top 30 features: AUC = 0.9583819939752696

In [9]:
# Get feature importances
feature_importances = clf.feature_importances_

# Create a DataFrame for feature importances
feature_importances_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
})

# Sort the DataFrame by importance
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

# Print the feature importances
print(feature_importances_df)

top_features = feature_importances_df['Feature'].head(65).tolist()

##30--> 0.963568372526153


   Feature  Importance
0       X2    0.069522
1       X3    0.041260
2       X4    0.036555
3       X5    0.032116
4       X6    0.027500
..     ...         ...
68     X70    0.000000
71     X73    0.000000
70     X72    0.000000
69     X71    0.000000
76     X78    0.000000

[77 rows x 2 columns]


In [10]:

X_train_top = X[top_features]
X_test_top = X[top_features]
X_testdata_top = X_testdata[top_features]

# Re-train the model with the top features
clf.fit(X_train_top,y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.0045, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None, min_child_weight=60, missing=nan,
              monotone_constraints=None, multi_strategy=None,
              n_estimators=12120, n_jobs=None, num_parallel_tree=None,
              random_state=42, ...)

In [25]:
y_pred_proba = clf.predict_proba(X_test_top)[:, 1]

# Calculate AUC
auc = roc_auc_score(y_test, y_pred_proba)
print(f"AUC: {auc}")

ValueError: Found input variables with inconsistent numbers of samples: [49225, 246122]

In [13]:
# Predict probabilities for the test set
y_pred_proba_testdata = clf.predict_proba(X_testdata_top)[:, 1]

# Create a DataFrame with RecordId and predicted probabilities
test_set['Y_probability'] = y_pred_proba_testdata

# Save the DataFrame to a CSV file
test_set[['RecordId', 'Y_probability']].to_csv('test_set_with_probabilities_nn.csv', index=False)

print("Probabilities saved to test_set_with_probabilities.csv")

import pandas as pd

# Read the generated CSV file
csv_file_path = 'test_set_with_probabilities.csv'
test_set_with_probabilities = pd.read_csv(csv_file_path)

# Get unique values in the Y_probability column
unique_y_probabilities = test_set_with_probabilities['Y_probability'].unique()

# Print the unique values
print(unique_y_probabilities)
print(test_set_with_probabilities)

Probabilities saved to test_set_with_probabilities.csv
[3.3025965e-03 1.4950908e-03 5.6551890e-05 ... 2.5839315e-04 3.0702833e-04
 1.6280190e-04]
        RecordId  Y_probability
0         300001       0.003303
1         300002       0.001495
2         300003       0.000057
3         300004       0.001137
4         300005       0.000211
...          ...            ...
105477    405478       0.000045
105478    405479       0.563237
105479    405480       0.000258
105480    405481       0.000307
105481    405482       0.000163

[105482 rows x 2 columns]
